An example for Tensor Decomposition

In [103]:
%load_ext autoreload
%autoreload 2
import cbx as cbx
from cbx.dynamics import CBO
from cbx.objectives import Rastrigin
import cbx.utils.termination as term
from cbx.scheduler import effective_sample_size


import numpy as np
import tensorly as tl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Define the tensor that we want to decompose

In [104]:
# Dimensions of the 3-way tensor
I, J, K = 10, 12, 14

# Create a random tensor
tensor = tl.tensor(np.random.random((I, J, K)))

# Define rank
rank = 5

In [105]:
tensor

array([[[0.2455526 , 0.1218777 , 0.20120624, ..., 0.4965727 ,
         0.7540427 , 0.12557037],
        [0.70138246, 0.13851716, 0.10552881, ..., 0.23223115,
         0.20972468, 0.02631908],
        [0.39200775, 0.39815091, 0.32636065, ..., 0.30790551,
         0.40036214, 0.23633321],
        ...,
        [0.42008973, 0.60417305, 0.27248502, ..., 0.39699895,
         0.32792294, 0.14289898],
        [0.36254645, 0.68801058, 0.44695512, ..., 0.13456593,
         0.99531381, 0.42792788],
        [0.95582404, 0.69291615, 0.42461499, ..., 0.38240277,
         0.19715776, 0.89422255]],

       [[0.65890714, 0.71839053, 0.76132804, ..., 0.16194523,
         0.31805053, 0.27495717],
        [0.4780809 , 0.84759011, 0.69402113, ..., 0.05418167,
         0.48959018, 0.62654427],
        [0.92194876, 0.49292669, 0.94504717, ..., 0.75943323,
         0.4679156 , 0.26370632],
        ...,
        [0.9149338 , 0.15750056, 0.10091374, ..., 0.10428115,
         0.42762386, 0.43276643],
        [0.4

Define the parameters

In [106]:
conf = {'alpha': 40.0,
        'dt': 0.1,
        'sigma': 1.,
        'lamda': 1.0,
        'batch_args':{
        'batch_size':200,
        'batch_partial': False},
        'tensor': tensor,
        'rank': rank,
        'd': 3,
        'term_criteria':[term.max_it_term(50)],
        'N': 100, # number of the particles
        'M': 100, # iterations
        'track_args': {'names':
                       ['update_norm',
                        'energy','x',
                        'consensus',
                        'drift']},}

Define the initial positions of the particles

In [107]:
particles = cbx.utils.init_particles(shape=(conf['M'], conf['N'], conf['d']), tensor=tensor, rank=conf['rank'], x_min=-1.0, x_max=1.0, delta=1.0, method="uniform")

In [114]:
particles = np.array([particles])

In [118]:
particles

IndexError: index 1 is out of bounds for axis 0 with size 1

Define the objective function (in objectives.py) and see the errors of the particles.

In [108]:
"""from cbx.objectives import TensorReconstructionError
error_func = TensorReconstructionError(tensor)

errors = []
for particle in particles:
    error = error_func(particle)
    errors.append(error)
    print(f"Reconstruction error: {error}")"""

'from cbx.objectives import TensorReconstructionError\nerror_func = TensorReconstructionError(tensor)\n\nerrors = []\nfor particle in particles:\n    error = error_func(particle)\n    errors.append(error)\n    print(f"Reconstruction error: {error}")'

In [119]:
from cbx.objectives import TensorReconstructionError
f = TensorReconstructionError(tensor)

Define the dynamic and scheduler

In [120]:
dyn = CBO(f, x=particles, noise='isotropic', f_dim='3D',
          **conf)
sched = effective_sample_size(name='alpha', eta = 0.1)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [111]:
dyn.optimize(sched=sched)

NameError: name 'dyn' is not defined